# UIDAI Biometric Data Analysis file 1

In [1]:
#importing necessary libraries
import pandas as pd

In [2]:
df = pd.read_csv('/content/demo_cleaned_final.csv')

In [3]:
df.shape

(1560734, 6)

In [4]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,uttar pradesh,gorakhpur,273213,49,529
1,01-03-2025,andhra pradesh,chittoor,517132,22,375
2,01-03-2025,gujarat,rajkot,360006,65,765
3,01-03-2025,andhra pradesh,srikakulam,532484,24,314
4,01-03-2025,rajasthan,udaipur,313801,45,785


In [ ]:
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [ ]:
df['district'] = df['district'].str.upper()
df['state'] = df['state'].str.upper()

In [5]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,uttar pradesh,gorakhpur,273213,49,529
1,01-03-2025,andhra pradesh,chittoor,517132,22,375
2,01-03-2025,gujarat,rajkot,360006,65,765
3,01-03-2025,andhra pradesh,srikakulam,532484,24,314
4,01-03-2025,rajasthan,udaipur,313801,45,785


In [ ]:
df['state']=df['state'].str.lower()
df['district']=df['district'].str.lower()

In [ ]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,haryana,mahendragarh,123029,280,577
1,01-03-2025,bihar,madhepura,852121,144,369
2,01-03-2025,bihar,bhojpur,802158,256,980
3,01-03-2025,tamil nadu,madurai,625514,271,815
4,01-03-2025,maharashtra,ratnagiri,416702,155,529


In [6]:
df.shape

(1560734, 6)

In [7]:
df_pre_final = df.copy()

In [8]:
df_pre_final.shape

(1560734, 6)

In [9]:
df_pre_final.columns

Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17',
       'demo_age_17_'],
      dtype='object')

In [10]:
import pandas as pd
import json

# Load your pincode JSON file
with open('pin-code-data.json', 'r') as f:
    pincode_data = json.load(f)

# Create a mapping of state to its valid pincodes
state_to_pincodes = {}
for pincode, info in pincode_data.items():
    state = info['state'].upper()

    if state not in state_to_pincodes:
        state_to_pincodes[state] = set()
    state_to_pincodes[state].add(pincode)

# Function to check if pincode belongs to state
def check_pincode_in_state(pincode, state):
    """
    Check if pincode belongs to the given state
    Returns: True if pincode exists for that state, False otherwise
    """
    pincode_str = str(pincode).strip()
    state_upper = state.upper().strip()

    if state_upper in state_to_pincodes:
        return pincode_str in state_to_pincodes[state_upper]
    else:
        return False

# Create new 'match' column

df_pre_final['match'] = df_pre_final.apply(
    lambda row: check_pincode_in_state(row['pincode'], row['state']),
    axis=1
)

# Display results
print(df_pre_final[['state', 'pincode', 'match']])

# Get summary
print("\nMatch Summary:")
print(df_pre_final['match'].value_counts())

# Show unmatched records
unmatched = df_pre_final[df_pre_final['match'] == False]
print(f"\nUnmatched Records: {len(unmatched)}")
print(unmatched[['state', 'pincode', 'match']])


                  state  pincode  match
0         uttar pradesh   273213   True
1        andhra pradesh   517132   True
2               gujarat   360006   True
3        andhra pradesh   532484   True
4             rajasthan   313801   True
...                 ...      ...    ...
1560729     west bengal   733128   True
1560730     west bengal   733150  False
1560731     west bengal   733211  False
1560732     west bengal   733212  False
1560733     west bengal   712701   True

[1560734 rows x 3 columns]

Match Summary:
match
True     1504480
False      56254
Name: count, dtype: int64

Unmatched Records: 56254
                 state  pincode  match
20          tamil nadu   603202  False
49       uttar pradesh   247670  False
57             gujarat   391155  False
74          chandigarh   160103  False
105            gujarat   384212  False
...                ...      ...    ...
1560689    west bengal   721167  False
1560723    west bengal   743385  False
1560730    west bengal   733150  

In [11]:
df_clean_after_state_check = df_pre_final[df_pre_final['match']==True]

In [13]:
df_clean_after_state_check.shape

(1504480, 7)

In [14]:
df_clean_after_state_check.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,match
0,01-03-2025,uttar pradesh,gorakhpur,273213,49,529,True
1,01-03-2025,andhra pradesh,chittoor,517132,22,375,True
2,01-03-2025,gujarat,rajkot,360006,65,765,True
3,01-03-2025,andhra pradesh,srikakulam,532484,24,314,True
4,01-03-2025,rajasthan,udaipur,313801,45,785,True


In [15]:
df_clean_after_state_check.to_csv('demo_final_cleaned_state_check.csv',index=False)

# Now Saving False data, where state not mapped to correct pincode

In [16]:
df_false_after_state_check = df_clean_after_state_check = df_pre_final[df_pre_final['match']==False]

In [18]:
df_false_after_state_check.shape

(56254, 7)

In [20]:
df_false_after_state_check.to_csv('demo_false_after_state_check.csv',index=False)